In [1]:
import numpy as np
import pandas as pd
import twosigmafunc

In [2]:
train = pd.read_hdf('train.h5')

In [3]:
train.sort_values(['id', 'timestamp'], inplace=True)
train.head()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y
131062,0,167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.007108
131895,0,168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001950
132728,0,169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017724
133561,0,170,-0.230583,0.488096,0.93592,0.028222,-0.083071,-0.240929,NaN,0.212425,...,0.727659,0.0,0.0,0.0,-0.160478,NaN,0.0,0.0,NaN,0.012934
134393,0,171,-0.230583,0.488096,0.93592,0.028222,-0.083071,-0.240929,NaN,0.212425,...,0.727659,0.0,0.0,0.0,-0.160478,NaN,0.0,0.0,NaN,-0.025229


In [4]:
origin_features = twosigmafunc.origin_features(train)
nan_features = twosigmafunc.addtional_features(origin_features, '_nan')
train['id_diff'] = train.id.diff()
initial_stage = train.loc[train.id_diff!=0]

In [5]:
initial_stage['nan_sum'] = initial_stage.isnull().sum(axis=1)

/usr/local/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [6]:
initial_stage.nan_sum.unique()

array([107, 106,  24,  22,  28,  38,  21,  30,  60,  27,  42,  51,  54,
        33,  26,  36,  35,  29,  23,  25,  20,  37,  47,  61,  43,  50,
        46,  34,  40,  31,  39,  32,  44,  63,  64,  82,  88,  56,  41,
        81,  59,  55,  52,  57,  89,  48,  58,  68,  49,  62,  66,  45,
        69,  65,  80])

In [7]:
for col, nan in zip(origin_features, nan_features):
    initial_stage[nan] = initial_stage[col].isnull()

/usr/local/lib/python3.4/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [8]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [10]:
# del train
inert = []
for n in range(2,10):
    km = KMeans(n_clusters=n, n_jobs=1, random_state=11, n_init=10)
    km.fit(initial_stage[nan_features + ['nan_sum']])
    score = silhouette_score(initial_stage[nan_features + ['nan_sum']], km.labels_)
    inert.append((n,score))

2
3
4
5
6
7
8
9


In [11]:
inert

[(2, 0.86891646043095638),
 (3, 0.82780727961783229),
 (4, 0.78999489664043965),
 (5, 0.77572947077190957),
 (6, 0.78254519477431117),
 (7, 0.77240612987586688),
 (8, 0.76355227039873197),
 (9, 0.7658268251075232)]

In [12]:
km = KMeans(n_clusters=5, n_jobs=1, random_state=11, n_init=20)
km.fit(initial_stage[nan_features + ['nan_sum']])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=20, n_jobs=1, precompute_distances='auto',
    random_state=11, tol=0.0001, verbose=0)

In [13]:
from collections import defaultdict
group_id = defaultdict(list)
for id_, group in zip(initial_stage.id, km.labels_):
    group_id[group].append(id_)

In [17]:
for key, gp in group_id.items():
    print(len(gp))

188
680
166
250
140


In [18]:
def label(id):
    if id in group_id[0]:
        return 'a'
    elif id in group_id[1]:
        return 'b'
    elif id in group_id[2]:
        return 'c'
    elif id in group_id[3]:
        return 'd'
    elif id in group_id[4]:
        return 'e'
    elif id in group_id[5]:
        return 'f'
    elif id in group_id[6]:
        return 'g'
    elif id in group_id[7]:
        return 'h'
    elif id in group_id[8]:
        return 'i'
    else:
        return 'n'

In [19]:
train.head()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y,id_diff
131062,0,167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.007108,NaN
131895,0,168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001950,0.0
132728,0,169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017724,0.0
133561,0,170,-0.230583,0.488096,0.93592,0.028222,-0.083071,-0.240929,NaN,0.212425,...,0.0,0.0,0.0,-0.160478,NaN,0.0,0.0,NaN,0.012934,0.0
134393,0,171,-0.230583,0.488096,0.93592,0.028222,-0.083071,-0.240929,NaN,0.212425,...,0.0,0.0,0.0,-0.160478,NaN,0.0,0.0,NaN,-0.025229,0.0


In [ ]:
label = train.id.map(label)

In [ ]:
label_dummy = pd.get_dummies(label, prefix='Label')
train.drop(['id', 'id_diff', 'timestamp'], axis=1, inplace=True)
train = pd.concat([train, label_dummy], axis=1)
train.fillna(train.median(), inplace=True)

In [20]:
for col, nan in zip(origin_features, nan_features):
    train[nan] = train[col].isnull()